In [4]:
import pandas as pd
import numpy as np

In [5]:
train = pd.read_csv("./dataset/train.csv")
test = pd.read_csv("./dataset/test.csv")

x_train = train.drop(columns=["Id", "label"]).to_numpy()
y_train = train["label"].to_numpy()[:, np.newaxis]

x_test = test.drop(columns=["Id"]).to_numpy()

print(x_train.shape)
print(y_train.shape)

print(x_test.shape)

(59597, 784)
(59597, 1)
(10403, 784)


In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import keras_tuner as kt

def build_model(hp):
    hp_layers = hp.Int('layers', min_value=6, max_value=20, step=2)
    hp_learning_rate = hp.Float(
      'learning_rate',
      min_value=1e-6,
      max_value=1e-2,
      sampling='LOG',
      default=1e-3
    )
    
    model = Sequential()
    
    model.add(Input(shape=(784,)))
    
    for i in range(hp_layers):
      model.add(Dense(hp.Int(f'units_{i}', min_value=32, max_value=2048, step=32), activation='relu'))
      model.add(Dropout(rate=hp.Float(
        f'dropout_rate_{i}',
        min_value=0.0,
        max_value=0.5,
        default=0.25,
        step=0.05,
      )))

    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate),
                    loss=SparseCategoricalCrossentropy(),
                    metrics=['accuracy'])

    return model

tuner = kt.Hyperband(build_model,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='tmp',
                     project_name='mnist')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner.search(x_train, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"Найкращі параметри:\n")
print(best_hps.values)

Reloading Tuner from tmp/mnist/tuner0.json
Найкращі параметри:

{'layers': 6, 'learning_rate': 3.796301284993931e-05, 'units_0': 416, 'dropout_rate_0': 0.05, 'units_1': 1792, 'dropout_rate_1': 0.05, 'units_2': 1056, 'dropout_rate_2': 0.30000000000000004, 'units_3': 1600, 'dropout_rate_3': 0.05, 'units_4': 1280, 'dropout_rate_4': 0.2, 'units_5': 736, 'dropout_rate_5': 0.15000000000000002, 'units_6': 160, 'dropout_rate_6': 0.4, 'units_7': 1632, 'dropout_rate_7': 0.45, 'units_8': 1568, 'dropout_rate_8': 0.15000000000000002, 'units_9': 1888, 'dropout_rate_9': 0.0, 'units_10': 2048, 'dropout_rate_10': 0.0, 'units_11': 1216, 'dropout_rate_11': 0.4, 'units_12': 1472, 'dropout_rate_12': 0.0, 'units_13': 384, 'dropout_rate_13': 0.30000000000000004, 'units_14': 1344, 'dropout_rate_14': 0.2, 'units_15': 1952, 'dropout_rate_15': 0.30000000000000004, 'units_16': 1632, 'dropout_rate_16': 0.2, 'units_17': 1984, 'dropout_rate_17': 0.35000000000000003, 'units_18': 1376, 'dropout_rate_18': 0.1, 'units_1

In [7]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(x_train, y_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1490/1490 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.6192 - loss: 3.4492 - val_accuracy: 0.9201 - val_loss: 0.2799
Epoch 2/50
1490/1490 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8788 - loss: 0.4240 - val_accuracy: 0.9396 - val_loss: 0.1999
Epoch 3/50
1490/1490 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9171 - loss: 0.2774 - val_accuracy: 0.9522 - val_loss: 0.1572
Epoch 4/50
1490/1490 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9401 - loss: 0.2010 - val_accuracy: 0.9595 - val_loss: 0.1362
Epoch 5/50
1490/1490 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9529 - loss: 0.1521 - val_accuracy: 0.9648 - val_loss: 0.1149
Epoch 6/50
1490/1490 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9604 - loss: 0.1250 - val_accuracy: 0.9652 - val_loss: 0.1247
Epoch 7/50
1490/1490 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9694 - loss: 0.0959 - val_accuracy: 0.9669 - val_loss: 0.1144
Epoch 8/50
1490/1490 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9770 - loss: 0.0745 - 

In [8]:
hypermodel = tuner.hypermodel.build(best_hps)

hypermodel.fit(x_train, y_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/49
1490/1490 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.6196 - loss: 3.4923 - val_accuracy: 0.9206 - val_loss: 0.2586
Epoch 2/49
1490/1490 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8726 - loss: 0.4355 - val_accuracy: 0.9413 - val_loss: 0.1964
Epoch 3/49
1490/1490 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9184 - loss: 0.2699 - val_accuracy: 0.9523 - val_loss: 0.1557
Epoch 4/49
1490/1490 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9388 - loss: 0.2014 - val_accuracy: 0.9579 - val_loss: 0.1340
Epoch 5/49
1490/1490 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9497 - loss: 0.1612 - val_accuracy: 0.9638 - val_loss: 0.1180
Epoch 6/49
1490/1490 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9601 - loss: 0.1318 - val_accuracy: 0.9666 - val_loss: 0.1151
Epoch 7/49
1490/1490 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9686 - loss: 0.1015 - val_accuracy: 0.9693 - val_loss: 0.1094
Epoch 8/49
1490/1490 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9755 - loss: 0.0782 - 

In [9]:
predictions = hypermodel.predict(x_test)

results = np.array(predictions).argmax(axis=1)

results

324/326 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

2025-03-08 09:49:26.396485: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_25', 304 bytes spill stores, 384 bytes spill loads

2025-03-08 09:49:26.773453: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_39', 288 bytes spill stores, 356 bytes spill loads

2025-03-08 09:49:27.041610: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_39', 68 bytes spill stores, 68 bytes spill loads

2025-03-08 09:49:27.333558: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_46', 60 bytes spill stores, 60 bytes spill loads

2025-03-08 09:49:27.466016: I external/local_xla/xla/stream_exec

326/326 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step


array([5, 3, 4, ..., 9, 1, 1])

In [10]:
ids = test["Id"].to_numpy()

result_df = pd.DataFrame({"Id": ids, "label": results}, columns=["Id", "label"])

In [11]:
result_df.to_csv("./output.csv", index=False)